# qtestNN

`qtest` is part of a Delaunay Triangulation algorithm.
This notebook explores how a neural network can be used to emulate its functionality.
The approach is to use `qtest` to build training and testing datasets.
Then additional intermediate values can be included in the NN to observe the improvement in accuracy.

I found that the circumcircle center cannot be determined by the net,
but all other intermediate values can be modeled by it.

## Preliminaries

Import the main packages.
Control tensorflow logging.

In [1]:
import tensorflow as tf
import numpy as np
print('tf version: {}'.format(tf.VERSION))

tf version: 1.7.0


In [2]:
from random import random
from time import time
from shutil import rmtree

## qtest

`qtest` in `pointset` operates on vertices.
This implementation operates on 4 points: the first three define the circumcircle,
the fourth is tested for inclusion in the circumcircle.
I've also broken down the steps into individual functions
so that meaningful intermediate values can be reproduced.

In [3]:
def dxdy(ix, iy, hx, hy):
    """The difference between two vectors"""
    return ix - hx, iy - hy

def norm(bx, by, cx, cy):
    """The norm used in calculating the circumcircle center"""
    return 2 * ( bx * cy - by * cx )

def circlexy(bx, by, cx, cy, d):
    """Return the circumcircle of the vectors and the origin"""
    ux = ( cy * (bx * bx + by * by) - by * (cx * cx + cy * cy) )
    uy = ( bx * (cx * cx + cy * cy) - cx * (bx * bx + by * by) )
    ux = ux / d
    uy = uy / d
    return ux, uy

def dist(x, y):
    """Return the distance between a point and the origin"""
    return int( (x * x + y * y)**0.5 * 100000) / 100000

def qtestxy(hx, hy, ix, iy, jx, jy, kx, ky):
    """Given 3 points, return True if the 4th point is not inside their circumcircle"""

    bx, by = dxdy(ix, iy, hx, hy)
    cx, cy = dxdy(jx, jy, hx, hy)
                                                                                                       
    n = norm(bx, by, cx, cy)

    ux, uy = circlexy(bx,  by, cx, cy, n)
    r = dist(ux, uy)

    x, y = dxdy(ux, uy, -hx, -hx)
    
    dx, dy = dxdy(kx, ky, x, y)
    d = dist(dx, dy)
    return True if d >= r else False

Check the function for a few cases.

In [4]:
print('.45: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .45)))
print('.49: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .49)))
print('.50: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .50)))
print('.51: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .51)))
print('.95: {}'.format(qtestxy(.4, .4, .5, .5, .6, .4, .5, .95)))

.45: False
.49: False
.50: False
.51: True
.95: True


## train

It's important to use `np` arrays for these structures so that the dimension of the tensors can be determined by tensorflow.

In [5]:
def train(epochs, ntrain, ntest, steps, layers, end, style=0, start=0, modeldir='qtestNN'):
    """train a data set, returning accuracy"""
    accuracy = []
    
    def samples(n):
        """returns an 8 vector of 4 random points"""
        res = []
        for _ in range(n):
            s = [ int(random()*100000)/100000 for _ in range(8) ]
            res.append(s)
        return np.array(res)

    trainingxs = samples(ntrain)
    testingxs = samples(ntest)

    def ysfor(xs):
        """return truth values using qtest"""
        return np.array([1 if qtestxy(h0, h1, i0, i1, j0, j1, k0, k1) else 0
                         for h0, h1, i0, i1, j0, j1, k0, k1 in xs])

    trainingys = ysfor(trainingxs)
    testingys = ysfor(testingxs)

    def parts0(hx, hy, ix, iy, jx, jy, kx, ky, pstlye=0):
        """return an array of intermediate calculations used in qtest"""

        bx, by = dxdy(ix, iy, hx, hy)
        cx, cy = dxdy(jx, jy, hx, hy)

                                                                                                        
        n = norm(bx, by, cx, cy)

        ux, uy = circlexy(bx,  by, cx, cy, n)
        r = dist(ux, uy)

        x, y = dxdy(ux, uy, -hx, -hx)
    
        dx, dy = dxdy(kx, ky, x, y)
        d = dist(dx, dy)
        v = 1.0 if d >= r else 0.0
        return [ bx, by, cx, cy, n, ux, uy, r, x, y, dx, dy, d, v ]
    
    def parts1(hx, hy, ix, iy, jx, jy, kx, ky):
        """return an array of distance attributes that might help find an accurate solution"""
        dx, dy = dxdy(kx, ky, hx, hy)
        kh = dist(dx, dy)
        
        dx, dy = dxdy(kx, ky, ix, iy)
        ki = dist(dx, dy)

        dx, dy = dxdy(kx, ky, jx, jy)
        kj = dist(dx, dy)

        dx, dy = dxdy(jx, jy, hx, hy)
        jh = dist(dx, dy)

        dx, dy = dxdy(jx, jy, ix, iy)
        ji = dist(dx, dy)

        dx, dy = dxdy(ix, iy, hx, hy)
        ih = dist(dx, dy)

        return [ kh, ki, kj, jh, ji, ih ]

    def parts2(hx, hy, ix, iy, jx, jy, kx, ky):
        """return an array of angle attributes that might help find an accurate solution"""

        def cos_angle(ax, ay, bx, by, cx, cy):
            # c2 = a2 + b2 - 2ab cos theta
            dx, dy = dxdy(ax, ay, bx, by)
            ab = dist(dx, dy)
            dx, dy = dxdy(cx, cy, bx, by)
            cb = dist(dx, dy)
            dx, dy = dxdy(ax, ay, cx, cy)
            ac = dist(dx, dy)
            return (ab*ab + cb*cb - ac*ac) / (2*ab*cb)

        hki = cos_angle(hx, hy, kx, ky, ix, iy)
        hkj = cos_angle(hx, hy, kx, ky, jx, jy)
        jki = cos_angle(jx, jy, kx, ky, ix, iy)

        hji = cos_angle(hx, hy, jx, jy, ix, iy)
        hjk = cos_angle(hx, hy, jx, jy, kx, ky)
        ijk = cos_angle(ix, iy, jx, jy, kx, ky)
        
        hij = cos_angle(hx, hy, ix, iy, jx, jy)
        hik = cos_angle(hx, hy, ix, iy, kx, ky)
        jik = cos_angle(jx, jy, ix, iy, kx, ky)

        ihj = cos_angle(ix, iy, hx, hy, jx, jy)
        ihk = cos_angle(ix, iy, hx, hy, kx, ky)
        jhk = cos_angle(jx, jy, hx, hy, kx, ky)

        return [ hki, hkj, jki,  hji, hjk, ijk,  hij, hik, jik,  ihj, ihk, jhk ]
    
    parts = parts0
    if style == 1:
        parts = parts1
    elif style == 2:
        parts = parts2


    def mixin(xs, start, end):
        """Update inputs with addition intermediate values"""
        ns = []
        for hx, hy, ix, iy, jx, jy, kx, ky in xs:
            ns.append([hx, hy, ix, iy, jx, jy, kx, ky] + parts(hx, hy, ix, iy, jx, jy, kx, ky)[start:end])
        return np.array(ns)
    
    trainingxs = mixin(trainingxs, start, end)
    testingxs = mixin(testingxs, start, end)
    
    try:
        rmtree(modeldir)
    except:
        pass
    
    verb = tf.logging.get_verbosity()
    tf.logging.set_verbosity(tf.logging.WARN)
    
    classifier = tf.estimator.DNNClassifier(
        feature_columns=[ tf.contrib.layers.real_valued_column("x", dimension = trainingxs.shape[1] ) ],
        hidden_units=layers,
        model_dir=modeldir,
        config=tf.estimator.RunConfig().replace(keep_checkpoint_max=1),
        n_classes=2)
    
    def prepx(x):
        return {
            'x': tf.constant(x)
        }
    
    def prepy(y):
        return tf.constant(y)
    
    def prep(x, y):
        return prepx(x), prepy(y)

    stime = time()
    for epoch in range(epochs):
        classifier.train(input_fn=lambda: prep(trainingxs, trainingys), steps=steps)
        accuracy_score = classifier.evaluate(input_fn=lambda: prep(testingxs, testingys), steps=1)["accuracy"]
        print(" {:5.3f}".format(accuracy_score), end='', flush=True)
        if epoch % 10 == 9:
            print(' {:03d} {:6.4f} hours'.format(1+epoch, (time()-stime)/3600))
        accuracy.append(accuracy_score)
        if accuracy_score > 0.95:
            break
    print('\n{:6.4f} hours'.format((time()-stime)/3600))
    tf.logging.set_verbosity(verb)

    return accuracy

For the first series of test, use a large 3 layer, 100 node neural net.
On each iteration, reduce the number of intermediate results to determine
which values are most helpful.
Since the intermediate values includes the answer, we will have a iteration with
accurate results. And since the final iteration doesn't include intermediate values,
we will have a baseline case as well.

In [6]:
for end in [14, 13, 12, 10, 8, 7, 5, 4, 0]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], end=end, style=0)

end: 14
Instructions for updating:
Use the retry module or similar alternatives.
 0.999
0.0023 hours
end: 13
 0.946 0.940 0.952
0.0063 hours
end: 12
 0.885 0.919 0.950 0.957
0.0086 hours
end: 10
 0.801 0.910 0.940 0.942 0.944 0.976
0.0124 hours
end: 8
 0.785 0.912 0.944 0.949 0.975
0.0103 hours
end: 7
 0.851 0.959
0.0040 hours
end: 5
 0.720 0.785 0.816 0.822 0.824 0.848 0.846 0.850 0.863 0.853 010 0.0203 hours
 0.855 0.860 0.849 0.861 0.860 0.861 0.866 0.861 0.864 0.863 020 0.0406 hours
 0.861 0.869 0.860 0.860 0.858 0.857 0.857 0.854 0.855 0.854 030 0.0612 hours
 0.853 0.854 0.854 0.856 0.860 0.861 0.863 0.862 0.860 0.863 040 0.0836 hours
 0.862 0.860 0.858 0.856 0.855 0.856 0.856 0.854 0.855 0.858 050 0.1058 hours
 0.858 0.858 0.857 0.856 0.856 0.857 0.855 0.858 0.857 0.855 060 0.1276 hours
 0.855 0.856 0.855 0.856 0.856 0.856 0.857 0.856 0.856 0.856 070 0.1493 hours
 0.858 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.858 0.858 080 0.1710 hours
 0.857 0.857 0.857 0.857 0.855 0.854 0.8

With 7 additional intermediate values the accuracy is high
and it drops significantly with 5 values.
Values 5 and 6 are ux, uy - the center of the circumcircle.

First, I'll check if a larger net might have with 5 intermediate values.

In [7]:
for end in [5]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100, 100, 100], end=end, style=0)

end: 5
 0.749 0.715 0.805 0.811 0.815 0.824 0.821 0.823 0.801 0.814 010 0.0316 hours
 0.839 0.833 0.820 0.832 0.826 0.832 0.827 0.829 0.826 0.823 020 0.0630 hours
 0.820 0.822 0.822 0.822 0.822 0.824 0.827 0.827 0.827 0.828 030 0.0942 hours
 0.828 0.828 0.828 0.828 0.830 0.829 0.829 0.829 0.829 0.828 040 0.1256 hours
 0.829 0.829 0.828 0.828 0.829 0.830 0.830 0.830 0.830 0.830 050 0.1571 hours
 0.830 0.830 0.830 0.830 0.830 0.830 0.830 0.830 0.829 0.829 060 0.1890 hours
 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 070 0.2203 hours
 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 080 0.2515 hours
 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 090 0.2827 hours
 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 0.829 100 0.3142 hours

0.3142 hours


There is an improvement, but it isn't very significant.
May be the net needs to be wider instead of deeper.

In [8]:
for end in [5]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[300, 300, 300], end=end, style=0)

end: 5
 0.748 0.754 0.779 0.797 0.808 0.827 0.849 0.854 0.836 0.845 010 0.0721 hours
 0.876 0.878 0.878 0.881 0.878 0.879 0.883 0.887 0.886 0.885 020 0.1442 hours
 0.882 0.883 0.880 0.880 0.878 0.879 0.878 0.880 0.881 0.882 030 0.2172 hours
 0.882 0.884 0.884 0.886 0.886 0.889 0.888 0.888 0.888 0.887 040 0.2929 hours
 0.887 0.888 0.888 0.888 0.888 0.888 0.887 0.887 0.888 0.887 050 0.3699 hours
 0.886 0.886 0.886 0.885 0.885 0.885 0.885 0.885 0.885 0.885 060 0.4421 hours
 0.885 0.885 0.885 0.886 0.886 0.886 0.886 0.886 0.886 0.886 070 0.5147 hours
 0.886 0.886 0.887 0.886 0.886 0.886 0.886 0.887 0.887 0.887 080 0.5871 hours
 0.887 0.887 0.887 0.887 0.887 0.887 0.887 0.887 0.887 0.887 090 0.6601 hours
 0.887 0.887 0.886 0.886 0.886 0.886 0.886 0.885 0.885 0.885 100 0.7327 hours

0.7327 hours


A wider net provides a smaller improvement.
So it doesn't seem to be a limitation of the size of the net.

Now I'll try to use distance features of the points as intermediate values.

In [9]:
for end in [6]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], end=end, style=1)

end: 6
 0.705 0.760 0.762 0.785 0.782 0.783 0.790 0.793 0.788 0.788 010 0.0206 hours
 0.796 0.792 0.805 0.796 0.809 0.817 0.809 0.817 0.815 0.802 020 0.0412 hours
 0.810 0.810 0.807 0.825 0.819 0.818 0.810 0.810 0.812 0.819 030 0.0616 hours
 0.817 0.805 0.814 0.814 0.813 0.811 0.815 0.814 0.817 0.810 040 0.0824 hours
 0.812 0.810 0.815 0.812 0.810 0.813 0.807 0.809 0.814 0.812 050 0.1029 hours
 0.806 0.815 0.814 0.815 0.808 0.809 0.812 0.815 0.813 0.812 060 0.1238 hours
 0.812 0.811 0.811 0.811 0.810 0.810 0.811 0.807 0.810 0.809 070 0.1446 hours
 0.809 0.810 0.810 0.809 0.810 0.810 0.809 0.810 0.812 0.809 080 0.1652 hours
 0.811 0.811 0.811 0.812 0.811 0.813 0.811 0.812 0.813 0.810 090 0.1865 hours
 0.814 0.813 0.812 0.810 0.812 0.813 0.811 0.812 0.812 0.812 100 0.2072 hours

0.2072 hours


No improvement with the distance features.

Now try angle features.

In [10]:
for end in [12]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], end=end, style=2)

end: 12
 0.732 0.803 0.820 0.828 0.840 0.847 0.848 0.857 0.857 0.863 010 0.0211 hours
 0.863 0.860 0.865 0.863 0.857 0.860 0.865 0.861 0.862 0.863 020 0.0427 hours
 0.862 0.862 0.861 0.860 0.859 0.859 0.863 0.861 0.861 0.860 030 0.0637 hours
 0.859 0.859 0.861 0.861 0.861 0.864 0.863 0.863 0.862 0.863 040 0.0848 hours
 0.862 0.862 0.862 0.862 0.862 0.861 0.861 0.861 0.860 0.860 050 0.1058 hours
 0.860 0.860 0.860 0.860 0.860 0.859 0.859 0.859 0.859 0.859 060 0.1271 hours
 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.858 0.858 070 0.1483 hours
 0.858 0.858 0.858 0.858 0.858 0.858 0.858 0.858 0.858 0.859 080 0.1696 hours
 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.859 090 0.1917 hours
 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.859 0.859 100 0.2153 hours

0.2153 hours


No improvement with these either.
But I'll do a quick check to see if the shape of the net makes any difference.

In [11]:
for end in [12]:
    print('end: {}'.format(end))
    train(epochs=1, ntrain=10000, ntest=1000, steps=100, layers=[300], end=end, style=2)

end: 12
 0.765
0.0017 hours


Now I'll focus on the circumcircle center.
I limit the additional features to only include the circumcircle center.

In [12]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100, 100, 100], start=end-2, end=end, style=0)

end: 7
 0.760 0.906 0.935 0.935 0.969
0.0107 hours


The converged quickly.
It seems that the circumcircle center is the key missing piece of information
that the net cannot determine on its own.

Now I'll see how small a net is needed with this expanded feature set.

In [13]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[100], start=end-2, end=end, style=0)

end: 7
 0.857 0.863 0.916 0.922 0.940 0.945 0.948 0.952
0.0080 hours


One lay works almost as well as 3 layers.

But now wide does it need to be?

In [14]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[50], start=end-2, end=end, style=0)

end: 7
 0.824 0.897 0.913 0.926 0.934 0.936 0.948 0.948 0.955
0.0078 hours


50 nodes works well ... may be fewer are needed?

In [15]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[25], start=end-2, end=end, style=0)

end: 7
 0.723 0.807 0.873 0.911 0.930 0.945 0.953
0.0053 hours


In [16]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[15], start=end-2, end=end, style=0)

end: 7
 0.730 0.849 0.885 0.907 0.913 0.924 0.926 0.931 0.932 0.931 010 0.0076 hours
 0.937 0.937 0.938 0.938 0.942 0.944 0.943 0.946 0.947 0.947 020 0.0152 hours
 0.949 0.950 0.952
0.0174 hours


In [17]:
for end in [7]:
    print('end: {}'.format(end))
    train(epochs=100, ntrain=10000, ntest=1000, steps=100, layers=[10], start=end-2, end=end, style=0)

end: 7
 0.663 0.671 0.699 0.723 0.740 0.756 0.773 0.804 0.840 0.865 010 0.0074 hours
 0.886 0.894 0.902 0.905 0.912 0.915 0.918 0.918 0.921 0.924 020 0.0146 hours
 0.926 0.928 0.932 0.932 0.939 0.942 0.941 0.941 0.941 0.944 030 0.0212 hours
 0.946 0.944 0.949 0.950 0.952
0.0246 hours


# Final Remarks

`qtest` is just one part of a complex algorithm for determining a Delaunay triangulation.
To train a net to emulate `qtest`, I created a perfectly correct, noiseless, training data set (as well as a test data set). I created a relatively large net and found that accuracy was limited to ~83%.
I tried a variety of additional features to improve the accuracy.
As expected it worked very well with lots of intermediate features.
But the accuracy dropped significantly when the circumcircle center was not included
in the intermediate features.
I did a number of simulations to confirm that the circle center was the critically
missing piece of information.
With the circle center, I was able to reduce the net size significantly.

This implies that the net cannot determine the circle center on its own.
It also suggests that other features, such as distances, can to modeled by the net.

Further work could look at creating a net that takes 3 points as input and
determine their circumcircle center.
Or confirming that other features can be readily modeled by a net.
